<a href="https://colab.research.google.com/github/Ayrsz/SignalAndSistemyProject/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Model CNN
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision


#Scientific computation
import numpy as np
import jax
import jax.numpy as jnp

#Plot and view
from matplotlib import pyplot as plt
import IPython as ipy
from IPython import display
from IPython.display import Audio

#Data manipulation
import os
import gc

#Audio manipulation
import librosa
import soundfile as sf

In [7]:
from zipfile import ZipFile
try:
    from google.colab import drive
    drive.mount("/content/drive", force_remount= True)
except:
    print("Código rodando localmente")

Mounted at /content/drive


In [8]:
class EfficientBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, expand_ratio=1):
        super(EfficientBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels * expand_ratio, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels * expand_ratio)
        self.conv2 = nn.Conv2d(in_channels * expand_ratio, out_channels, kernel_size, stride, padding, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.expand_ratio = expand_ratio

    def forward(self, x):
        identity = x

        out = F.relu6(self.bn1(self.conv1(x)))
        out = F.relu6(self.bn2(self.conv2(out)))

        if identity.shape[1] == out.shape[1]:
            out += identity
        return out

class EfficientNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(EfficientNet, self).__init__()
        self.stem_conv = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False)
        self.stem_bn = nn.BatchNorm2d(32)
        self.blocks = nn.Sequential(
            EfficientBlock(32, 16, expand_ratio=1),
            EfficientBlock(16, 24, expand_ratio=6),
            EfficientBlock(24, 40, expand_ratio=6),
            EfficientBlock(40, 80, expand_ratio=6),
            EfficientBlock(80, 112, expand_ratio=6),
            EfficientBlock(112, 192, expand_ratio=6),
            EfficientBlock(192, 320, expand_ratio=6)
        )
        self.global_pool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = F.relu6(self.stem_bn(self.stem_conv(x)))
        x = self.blocks(x)
        x = self.global_pool(x)
        x = torch.flatten(x, 1)

        return x # Return a vector of pool,

class Genre_Classification(EfficientNet):
    def __init__(self, feature_size = 56, num_classes = 10):
        super(Genre_Classification, self).__init__()

    def forward(self, img_x, carac_x):
        cnn_response = super(Genre_Classification, self).forward(self, x = img_x)

